In [2]:
# Imports and Data Loading
import pandas as pd
import numpy as np

# Load the hitter data
df = pd.read_csv('hitters/all_hitter_data.csv')
print(f"Total rows: {len(df)}")
print(f"\nColumns available: {list(df.columns)}")

Total rows: 31151

Columns available: ['Unnamed: 0', 'come', 'link', 'player_state', 'high_school', 'class', 'primary_position', 'commitment', 'age', 'positions', 'height', 'weight', 'throwing_hand', 'hitting_handedness', 'hand_speed_max', 'bat_speed_max', 'rot_acc_max', 'hard_hit_p', 'sixty_time', 'thirty_time', 'ten_yard_time', 'run_speed_max', 'exit_velo_max', 'exit_velo_avg', 'distance_max', 'sweet_spot_p', 'inf_velo', 'of_velo', 'c_velo', 'pop_time', 'player_region', 'player_section_of_region', 'confidence', 'conference', 'division', 'college_location', 'conf_short', 'committment_group', 'position_velo', 'three_section_commit_group']


## Define Division Groups

Filter data into the four division groups:
- **P4**: Power 4 D1 conferences
- **Non-P4 D1**: Non-Power 4 D1 conferences  
- **D2**: Division 2
- **D3**: Division 3

Exclude JC and NAIA from the Non D1 group.

In [3]:
# Create division group filters
df_p4 = df[df['three_section_commit_group'] == 'Power 4 D1'].copy()
df_non_p4_d1 = df[df['three_section_commit_group'] == 'Non P4 D1'].copy()

# For D2 and D3, filter from "Non D1" and use committment_group
# Exclude JC and NAIA
df_non_d1 = df[df['three_section_commit_group'] == 'Non D1'].copy()
df_d2 = df_non_d1[df_non_d1['committment_group'] == 'D2'].copy()
df_d3 = df_non_d1[df_non_d1['committment_group'] == 'D3'].copy()

print("Division group sizes:")
print(f"  P4:        {len(df_p4):,} players")
print(f"  Non-P4 D1: {len(df_non_p4_d1):,} players")
print(f"  D2:        {len(df_d2):,} players")
print(f"  D3:        {len(df_d3):,} players")
print(f"  Total:     {len(df_p4) + len(df_non_p4_d1) + len(df_d2) + len(df_d3):,} players (excluding JC/NAIA)")

Division group sizes:
  P4:        2,603 players
  Non-P4 D1: 5,630 players
  D2:        4,176 players
  D3:        7,103 players
  Total:     19,512 players (excluding JC/NAIA)


## Define Position Groups for Velocity Filtering

For position-specific velocities, we need to filter by primary_position:
- **Infielders** (inf_velo): SS, 3B, 1B, 2B, MIF, IF
- **Outfielders** (of_velo): OF, CF, RF, LF
- **Catchers** (c_velo, pop_time): C

In [4]:
# Position group definitions
INFIELDER_POSITIONS = ['SS', '3B', '1B', '2B', 'MIF', 'IF']
OUTFIELDER_POSITIONS = ['OF', 'CF', 'RF', 'LF']
CATCHER_POSITIONS = ['C']

# Reasonable bounds for filtering outliers (data entry errors)
BOUNDS = {
    'exit_velo_max': (60, 120),   # mph
    'sixty_time': (5.5, 9.0),     # seconds
    'height': (60, 84),           # inches (5'0" to 7'0")
    'weight': (120, 320),         # lbs
    'inf_velo': (50, 100),        # mph
    'of_velo': (50, 105),         # mph
    'c_velo': (50, 95),           # mph
    'pop_time': (1.5, 2.8),       # seconds
}

def filter_outliers(series, bounds):
    """Filter values outside reasonable bounds."""
    low, high = bounds
    return series[(series >= low) & (series <= high)]

def calculate_benchmarks(division_df):
    """
    Calculate mean and std for all stats needed for a division group.
    Filters outliers using BOUNDS before calculating.
    """
    results = {}
    
    # Universal stats (all players)
    universal_stats = {
        'exit_velo': 'exit_velo_max',
        'sixty_time': 'sixty_time',
        'height': 'height',
        'weight': 'weight'
    }
    
    for stat_name, col_name in universal_stats.items():
        values = division_df[col_name].dropna()
        values = filter_outliers(values, BOUNDS[col_name])
        results[stat_name] = {
            'mean': round(values.mean(), 2),
            'std': round(values.std(), 2),
            'count': len(values)
        }
    
    # Infielder velocity (only for infielders)
    inf_df = division_df[division_df['primary_position'].isin(INFIELDER_POSITIONS)]
    values = filter_outliers(inf_df['inf_velo'].dropna(), BOUNDS['inf_velo'])
    results['inf_velo'] = {
        'mean': round(values.mean(), 2),
        'std': round(values.std(), 2),
        'count': len(values)
    }
    
    # Outfielder velocity (only for outfielders)
    of_df = division_df[division_df['primary_position'].isin(OUTFIELDER_POSITIONS)]
    values = filter_outliers(of_df['of_velo'].dropna(), BOUNDS['of_velo'])
    results['of_velo'] = {
        'mean': round(values.mean(), 2),
        'std': round(values.std(), 2),
        'count': len(values)
    }
    
    # Catcher velocity and pop time (only for catchers)
    c_df = division_df[division_df['primary_position'].isin(CATCHER_POSITIONS)]
    
    values = filter_outliers(c_df['c_velo'].dropna(), BOUNDS['c_velo'])
    results['c_velo'] = {
        'mean': round(values.mean(), 2),
        'std': round(values.std(), 2),
        'count': len(values)
    }
    
    values = filter_outliers(c_df['pop_time'].dropna(), BOUNDS['pop_time'])
    results['pop_time'] = {
        'mean': round(values.mean(), 2),
        'std': round(values.std(), 2),
        'count': len(values)
    }
    
    return results

print("Benchmark calculation function defined with outlier filtering.")

Benchmark calculation function defined with outlier filtering.


## Calculate Benchmarks for Each Division

In [5]:
# Calculate benchmarks for each division
benchmarks = {
    'P4': calculate_benchmarks(df_p4),
    'Non-P4 D1': calculate_benchmarks(df_non_p4_d1),
    'D2': calculate_benchmarks(df_d2),
    'D3': calculate_benchmarks(df_d3)
}

# Display results in a readable format
for division, stats in benchmarks.items():
    print(f"\n{'='*60}")
    print(f"  {division} BENCHMARKS")
    print(f"{'='*60}")
    for stat_name, values in stats.items():
        print(f"  {stat_name:12} | mean: {values['mean']:6.2f} | std: {values['std']:5.2f} | n={values['count']:,}")


  P4 BENCHMARKS
  exit_velo    | mean:  95.40 | std:  5.97 | n=1,956
  sixty_time   | mean:   7.02 | std:  0.34 | n=1,866
  height       | mean:  72.65 | std:  2.25 | n=2,593
  weight       | mean: 187.32 | std: 19.04 | n=2,597
  inf_velo     | mean:  84.66 | std:  5.29 | n=1,014
  of_velo      | mean:  86.94 | std:  5.57 | n=527
  c_velo       | mean:  79.02 | std:  3.90 | n=353
  pop_time     | mean:   1.99 | std:  0.10 | n=354

  Non-P4 D1 BENCHMARKS
  exit_velo    | mean:  93.40 | std:  5.58 | n=4,450
  sixty_time   | mean:   7.10 | std:  0.34 | n=4,300
  height       | mean:  72.11 | std:  2.22 | n=5,608
  weight       | mean: 182.71 | std: 18.66 | n=5,617
  inf_velo     | mean:  82.94 | std:  5.01 | n=2,215
  of_velo      | mean:  85.53 | std:  4.94 | n=1,212
  c_velo       | mean:  77.54 | std:  3.87 | n=922
  pop_time     | mean:   2.00 | std:  0.10 | n=922

  D2 BENCHMARKS
  exit_velo    | mean:  91.00 | std:  5.44 | n=3,472
  sixty_time   | mean:   7.25 | std:  0.35 | n=3,31

## Format as DIVISION_BENCHMARKS Dict

This output can be copied directly into `constants.py`:

In [6]:
# Generate copy-pastable DIVISION_BENCHMARKS dict
print("DIVISION_BENCHMARKS: Dict[str, Dict[str, Dict[str, float]]] = {")
for division, stats in benchmarks.items():
    print(f'    "{division}": {{')
    stat_items = list(stats.items())
    for i, (stat_name, values) in enumerate(stat_items):
        comma = "," if i < len(stat_items) - 1 else ""
        print(f'        "{stat_name}": {{"mean": {values["mean"]}, "std": {values["std"]}}}{comma}')
    print("    },")
print("}")

DIVISION_BENCHMARKS: Dict[str, Dict[str, Dict[str, float]]] = {
    "P4": {
        "exit_velo": {"mean": 95.4, "std": 5.97},
        "sixty_time": {"mean": 7.02, "std": 0.34},
        "height": {"mean": 72.65, "std": 2.25},
        "weight": {"mean": 187.32, "std": 19.04},
        "inf_velo": {"mean": 84.66, "std": 5.29},
        "of_velo": {"mean": 86.94, "std": 5.57},
        "c_velo": {"mean": 79.02, "std": 3.9},
        "pop_time": {"mean": 1.99, "std": 0.1}
    },
    "Non-P4 D1": {
        "exit_velo": {"mean": 93.4, "std": 5.58},
        "sixty_time": {"mean": 7.1, "std": 0.34},
        "height": {"mean": 72.11, "std": 2.22},
        "weight": {"mean": 182.71, "std": 18.66},
        "inf_velo": {"mean": 82.94, "std": 5.01},
        "of_velo": {"mean": 85.53, "std": 4.94},
        "c_velo": {"mean": 77.54, "std": 3.87},
        "pop_time": {"mean": 2.0, "std": 0.1}
    },
    "D2": {
        "exit_velo": {"mean": 91.0, "std": 5.44},
        "sixty_time": {"mean": 7.25, "std": 0.

## Compare to Original Placeholders

Let's see how the calculated values differ from the placeholders:

In [7]:
# Original placeholder values
PLACEHOLDERS = {
    "P4": {
        "exit_velo": {"mean": 96.0, "std": 4.0},
        "sixty_time": {"mean": 6.75, "std": 0.18},
        "inf_velo": {"mean": 87.0, "std": 3.5},
        "of_velo": {"mean": 90.0, "std": 3.5},
        "c_velo": {"mean": 83.0, "std": 3.0},
        "pop_time": {"mean": 1.95, "std": 0.07},
        "height": {"mean": 73.0, "std": 2.5},
        "weight": {"mean": 195.0, "std": 18.0},
    },
    "Non-P4 D1": {
        "exit_velo": {"mean": 92.0, "std": 4.5},
        "sixty_time": {"mean": 7.0, "std": 0.20},
        "inf_velo": {"mean": 83.0, "std": 3.5},
        "of_velo": {"mean": 86.0, "std": 3.5},
        "c_velo": {"mean": 80.0, "std": 3.0},
        "pop_time": {"mean": 2.02, "std": 0.08},
        "height": {"mean": 72.0, "std": 2.5},
        "weight": {"mean": 185.0, "std": 17.0},
    },
    "D2": {
        "exit_velo": {"mean": 88.0, "std": 5.0},
        "sixty_time": {"mean": 7.15, "std": 0.22},
        "inf_velo": {"mean": 80.0, "std": 3.5},
        "of_velo": {"mean": 83.0, "std": 3.5},
        "c_velo": {"mean": 77.0, "std": 3.0},
        "pop_time": {"mean": 2.08, "std": 0.09},
        "height": {"mean": 71.5, "std": 2.5},
        "weight": {"mean": 180.0, "std": 16.0},
    },
    "D3": {
        "exit_velo": {"mean": 84.0, "std": 5.5},
        "sixty_time": {"mean": 7.30, "std": 0.25},
        "inf_velo": {"mean": 77.0, "std": 3.5},
        "of_velo": {"mean": 80.0, "std": 3.5},
        "c_velo": {"mean": 74.0, "std": 3.0},
        "pop_time": {"mean": 2.15, "std": 0.10},
        "height": {"mean": 71.0, "std": 2.5},
        "weight": {"mean": 175.0, "std": 15.0},
    }
}

# Compare
print("COMPARISON: Placeholder vs Calculated")
print("=" * 80)
print(f"{'Division':<12} {'Stat':<12} {'Placeholder Mean':>16} {'Calculated Mean':>16} {'Diff':>8}")
print("-" * 80)

for division in ['P4', 'Non-P4 D1', 'D2', 'D3']:
    for stat in ['exit_velo', 'sixty_time', 'inf_velo', 'of_velo', 'c_velo', 'pop_time', 'height', 'weight']:
        placeholder_mean = PLACEHOLDERS[division][stat]['mean']
        calculated_mean = benchmarks[division][stat]['mean']
        diff = calculated_mean - placeholder_mean
        
        # Highlight significant differences
        marker = "***" if abs(diff) > 2 else ""
        print(f"{division:<12} {stat:<12} {placeholder_mean:>16.2f} {calculated_mean:>16.2f} {diff:>+8.2f} {marker}")
    print("-" * 80)

COMPARISON: Placeholder vs Calculated
Division     Stat         Placeholder Mean  Calculated Mean     Diff
--------------------------------------------------------------------------------
P4           exit_velo               96.00            95.40    -0.60 
P4           sixty_time               6.75             7.02    +0.27 
P4           inf_velo                87.00            84.66    -2.34 ***
P4           of_velo                 90.00            86.94    -3.06 ***
P4           c_velo                  83.00            79.02    -3.98 ***
P4           pop_time                 1.95             1.99    +0.04 
P4           height                  73.00            72.65    -0.35 
P4           weight                 195.00           187.32    -7.68 ***
--------------------------------------------------------------------------------
Non-P4 D1    exit_velo               92.00            93.40    +1.40 
Non-P4 D1    sixty_time               7.00             7.10    +0.10 
Non-P4 D1    inf_ve

## Pitcher division benchmarks
Compute benchmarks for P4, Non-P4 D1, D2, D3 using pitcher dataset.


In [8]:
# Load pitcher data
pitchers_path = '/Users/ryankolodziejczyk/Documents/AI Baseball Recruitment/code/backend/data/pitchers/pitchers_data_clean.csv'
df_p = pd.read_csv(pitchers_path)
print(f'Pitcher rows: {len(df_p)}')
print(df_p.columns)


Pitcher rows: 18343
Index(['primary_position', 'height', 'weight', 'throwing_hand',
       'FastballVelocity (max)', 'FastballVelo Range',
       'FastballSpin Rate (avg)', 'Changeup Velo Range',
       'Changeup Spin Rate (avg)', 'Curveball Velo Range',
       'Curveball Spin Rate (avg)', 'Slider Velo Range',
       'Slider Spin Rate (avg)', 'Region', 'group', 'num_pitches',
       'fb_ch_velo_diff', 'fb_cb_velo_diff', 'fb_sl_velo_diff',
       'FastballVelocity (max)_missing', 'FastballVelo Range_missing',
       'FastballSpin Rate (avg)_missing', 'Changeup Velo Range_missing',
       'Changeup Spin Rate (avg)_missing', 'Curveball Velo Range_missing',
       'Curveball Spin Rate (avg)_missing', 'Slider Velo Range_missing',
       'Slider Spin Rate (avg)_missing'],
      dtype='object')


In [9]:
# Pitcher division group filters
df_p4_p = df_p[df_p['group'] == 'Power 4'].copy()
df_non_p4_d1_p = df_p[df_p['group'].isin(['Mid Major', 'Low Major'])].copy()
df_d2_p = df_p[df_p['group'] == 'D2'].copy()
df_d3_p = df_p[df_p['group'] == 'D3'].copy()

print('Pitcher division group sizes:')
print(f'  P4:        {len(df_p4_p):,} players')
print(f'  Non-P4 D1: {len(df_non_p4_d1_p):,} players')
print(f'  D2:        {len(df_d2_p):,} players')
print(f'  D3:        {len(df_d3_p):,} players')
print(f'  Total:     {len(df_p4_p) + len(df_non_p4_d1_p) + len(df_d2_p) + len(df_d3_p):,} players')


Pitcher division group sizes:
  P4:        2,253 players
  Non-P4 D1: 4,150 players
  D2:        2,295 players
  D3:        3,542 players
  Total:     12,240 players


In [10]:
# Bounds for pitcher metrics (to filter obvious outliers)
PITCHER_BOUNDS = {
    'height': (60, 84),
    'weight': (120, 320),
    'FastballVelo Range': (60, 105),
    'FastballVelocity (max)': (60, 105),
    'FastballSpin Rate (avg)': (1200, 3500),
    'Changeup Velo Range': (60, 95),
    'Changeup Spin Rate (avg)': (800, 3200),
    'Curveball Velo Range': (55, 95),
    'Curveball Spin Rate (avg)': (1200, 3500),
    'Slider Velo Range': (60, 100),
    'Slider Spin Rate (avg)': (1200, 3500),
}

def calculate_benchmarks_pitchers(division_df):
    stats = {}
    for stat, bounds in PITCHER_BOUNDS.items():
        if stat not in division_df.columns:
            continue
        series = division_df[stat].dropna()
        low, high = bounds
        series = series[(series >= low) & (series <= high)]
        if len(series) == 0:
            stats[stat] = {'mean': float('nan'), 'std': float('nan'), 'count': 0}
        else:
            stats[stat] = {
                'mean': float(series.mean()),
                'std': float(series.std()),
                'count': int(series.count())
            }
    return stats


In [11]:
# Calculate pitcher benchmarks
pitcher_benchmarks = {
    'P4': calculate_benchmarks_pitchers(df_p4_p),
    'Non-P4 D1': calculate_benchmarks_pitchers(df_non_p4_d1_p),
    'D2': calculate_benchmarks_pitchers(df_d2_p),
    'D3': calculate_benchmarks_pitchers(df_d3_p),
}

# Display results
for division, stats in pitcher_benchmarks.items():
    print('')
    print('='*60)
    print(f'  {division} PITCHER BENCHMARKS')
    print('='*60)
    for stat_name, values in stats.items():
        print('  {name:26} | mean: {mean:6.2f} | std: {std:6.2f} | n={count:,}'.format(
            name=stat_name, mean=values['mean'], std=values['std'], count=values['count']
        ))



  P4 PITCHER BENCHMARKS
  height                     | mean:  74.40 | std:   2.19 | n=2,252
  weight                     | mean: 192.69 | std:  19.50 | n=2,251
  FastballVelo Range         | mean:  88.39 | std:   3.62 | n=1,974
  FastballVelocity (max)     | mean:  90.47 | std:   3.63 | n=1,770
  FastballSpin Rate (avg)    | mean: 2187.07 | std: 194.27 | n=1,303
  Changeup Velo Range        | mean:  79.49 | std:   4.27 | n=1,736
  Changeup Spin Rate (avg)   | mean: 1764.20 | std: 265.67 | n=1,128
  Curveball Velo Range       | mean:  74.22 | std:   4.14 | n=1,792
  Curveball Spin Rate (avg)  | mean: 2221.73 | std: 307.94 | n=1,016
  Slider Velo Range          | mean:  77.27 | std:   4.17 | n=1,295
  Slider Spin Rate (avg)     | mean: 2267.83 | std: 295.67 | n=870

  Non-P4 D1 PITCHER BENCHMARKS
  height                     | mean:  73.69 | std:   2.24 | n=4,149
  weight                     | mean: 187.89 | std:  19.33 | n=4,144
  FastballVelo Range         | mean:  85.92 | std:   3.40

In [12]:
# Copy-pastable dict for pitcher benchmarks
print('PITCHER_DIVISION_BENCHMARKS = {')
for division, stats in pitcher_benchmarks.items():
    print('    "{}": {{'.format(division))
    stat_items = list(stats.items())
    for i, (stat_name, values) in enumerate(stat_items):
        comma = ',' if i < len(stat_items) - 1 else ''
        print('        "{}": {{"mean": {}, "std": {}}}{}'.format(
            stat_name, values['mean'], values['std'], comma
        ))
    print('    },')
print('}')


PITCHER_DIVISION_BENCHMARKS = {
    "P4": {
        "height": {"mean": 74.39609236234459, "std": 2.1880379950510553},
        "weight": {"mean": 192.69471346068414, "std": 19.49675878578776},
        "FastballVelo Range": {"mean": 88.38520770010133, "std": 3.617650321602161},
        "FastballVelocity (max)": {"mean": 90.46785310734465, "std": 3.634686301738456},
        "FastballSpin Rate (avg)": {"mean": 2187.0721412125863, "std": 194.26862967348072},
        "Changeup Velo Range": {"mean": 79.48842165898617, "std": 4.265934851670926},
        "Changeup Spin Rate (avg)": {"mean": 1764.2039007092199, "std": 265.6706688018309},
        "Curveball Velo Range": {"mean": 74.21941964285715, "std": 4.13795495904926},
        "Curveball Spin Rate (avg)": {"mean": 2221.732283464567, "std": 307.935431113531},
        "Slider Velo Range": {"mean": 77.26903474903474, "std": 4.167591186879467},
        "Slider Spin Rate (avg)": {"mean": 2267.8275862068967, "std": 295.6677102175398}
    },
    "No

In [19]:
PLACEHOLDERS_PITCHERS = {
    "P4": {
        "height": {"mean": 74.40, "std": 2.19},
        "weight": {"mean": 192.69, "std": 19.50},
        "FastballVelo Range": {"mean": 88.39, "std": 3.62},
        "FastballVelocity (max)": {"mean": 90.47, "std": 3.63},
        "FastballSpin Rate (avg)": {"mean": 2187.07, "std": 194.27},
        "Changeup Velo Range": {"mean": 79.49, "std": 4.27},
        "Changeup Spin Rate (avg)": {"mean": 1764.20, "std": 265.67},
        "Curveball Velo Range": {"mean": 74.22, "std": 4.14},
        "Curveball Spin Rate (avg)": {"mean": 2221.73, "std": 307.94},
        "Slider Velo Range": {"mean": 77.27, "std": 4.17},
        "Slider Spin Rate (avg)": {"mean": 2267.83, "std": 295.67},
    },
    "Non-P4 D1": {
        "height": {"mean": 73.69, "std": 2.24},
        "weight": {"mean": 187.89, "std": 19.33},
        "FastballVelo Range": {"mean": 85.92, "std": 3.40},
        "FastballVelocity (max)": {"mean": 87.88, "std": 3.39},
        "FastballSpin Rate (avg)": {"mean": 2137.07, "std": 177.43},
        "Changeup Velo Range": {"mean": 77.55, "std": 4.08},
        "Changeup Spin Rate (avg)": {"mean": 1710.87, "std": 262.72},
        "Curveball Velo Range": {"mean": 72.56, "std": 3.84},
        "Curveball Spin Rate (avg)": {"mean": 2149.25, "std": 281.01},
        "Slider Velo Range": {"mean": 75.09, "std": 4.00},
        "Slider Spin Rate (avg)": {"mean": 2191.69, "std": 277.93},
    },
    "D2": {
        "height": {"mean": 73.10, "std": 2.33},
        "weight": {"mean": 183.80, "std": 20.86},
        "FastballVelo Range": {"mean": 82.72, "std": 3.70},
        "FastballVelocity (max)": {"mean": 84.53, "std": 3.75},
        "FastballSpin Rate (avg)": {"mean": 2048.72, "std": 188.77},
        "Changeup Velo Range": {"mean": 74.95, "std": 4.07},
        "Changeup Spin Rate (avg)": {"mean": 1650.76, "std": 257.34},
        "Curveball Velo Range": {"mean": 70.40, "std": 3.95},
        "Curveball Spin Rate (avg)": {"mean": 2061.99, "std": 278.57},
        "Slider Velo Range": {"mean": 72.58, "std": 3.89},
        "Slider Spin Rate (avg)": {"mean": 2112.39, "std": 260.86},
    },
    "D3": {
        "height": {"mean": 72.47, "std": 2.35},
        "weight": {"mean": 179.20, "std": 21.22},
        "FastballVelo Range": {"mean": 80.20, "std": 3.86},
        "FastballVelocity (max)": {"mean": 81.87, "std": 3.83},
        "FastballSpin Rate (avg)": {"mean": 1988.63, "std": 188.67},
        "Changeup Velo Range": {"mean": 72.99, "std": 4.00},
        "Changeup Spin Rate (avg)": {"mean": 1600.67, "std": 249.39},
        "Curveball Velo Range": {"mean": 68.44, "std": 4.04},
        "Curveball Spin Rate (avg)": {"mean": 1999.35, "std": 266.94},
        "Slider Velo Range": {"mean": 70.59, "std": 3.83},
        "Slider Spin Rate (avg)": {"mean": 2036.34, "std": 254.21},
    },
}

# Compare
print("COMPARISON: Placeholder vs Calculated")
print("=" * 80)
print(f"{'Division':<12} {'Stat':<12} {'Calculated Mean':>16}")
print("-" * 80)

for division in ['P4', 'Non-P4 D1', 'D2', 'D3']:
    for stat in ['FastballVelo Range', 'FastballVelocity (max)', 'FastballSpin Rate (avg)', 'Changeup Velo Range', 'Curveball Velo Range', 'Slider Velo Range']:
        calculated_mean = pitcher_benchmarks[division][stat]['mean']
        print(f"{division:<12} {stat:<12} {calculated_mean:>16.2f}")

COMPARISON: Placeholder vs Calculated
Division     Stat          Calculated Mean
--------------------------------------------------------------------------------
P4           FastballVelo Range            88.39
P4           FastballVelocity (max)            90.47
P4           FastballSpin Rate (avg)          2187.07
P4           Changeup Velo Range            79.49
P4           Curveball Velo Range            74.22
P4           Slider Velo Range            77.27
Non-P4 D1    FastballVelo Range            85.92
Non-P4 D1    FastballVelocity (max)            87.88
Non-P4 D1    FastballSpin Rate (avg)          2137.07
Non-P4 D1    Changeup Velo Range            77.55
Non-P4 D1    Curveball Velo Range            72.56
Non-P4 D1    Slider Velo Range            75.09
D2           FastballVelo Range            82.72
D2           FastballVelocity (max)            84.53
D2           FastballSpin Rate (avg)          2048.72
D2           Changeup Velo Range            74.95
D2           Curveball